In [6]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'module')))
from openTable import *
from filepath import *
from preprocessing import preprocessing_text as pre

import warnings
warnings.filterwarnings('ignore')

from re import sub
import pandas as pd

# import spacy
from spacy.lang.id import Indonesian,stop_words
nlp = Indonesian()  # use directly
stopwords = stop_words.STOP_WORDS 
stopwords |= {"nya","jurusan","jurus","the","of"}

def preprocessing(text):
    text = pre.remove_tag(text) #Remove Tag
    text = pre.lower(text) #Lower
    text = pre.remove_link(text) #Remove Link
    text = pre.alphabet_only(text) #Get Alphabet
    text = sub(r'sobat pintar','',text) # sorry:(
    text = pre.remove_whitespace(text) #Remove Whitespace
    text = [token.text for token in nlp(text)] #Token
    text = pre.slang(text)
    text = [token.lemma_ for token in nlp(text) if token.lemma_ not in stopwords] #Lemma & stopword
    
    return text

def get_data():
    data = open_table(['entryId','content'],'BlogsEntry')
    
    return data

def cleaning():
    #get data
    data = get_data()
    data = rename_column(data,{0:'entryId', 1:'content'})
    data.content = data.content.apply(preprocessing)
    
    return data

In [7]:
data = cleaning()
display(data.head())
print(len(data.entryId.unique()))

#Save
data.to_csv('../data/data_berita_clean.csv', sep=',',header=data.columns.tolist(),index=None)

,entryId,content
0,204420,"[kakak, blog, membahas, tips, pembahasan, stan..."
1,288433,"[peluang, snmptn, nilai, rapormu, turun, ombak..."
2,288455,"[tertarik, snmptn, hmm, akibat, nilai, rapor, ..."
3,288478,"[mendengar, belajar, muncul, dikepalamu, kesal..."
4,291921,"[masuk, kelas, sesuai, jadwal, pulang, sekolah..."


757


In [7]:
# #Count Tag
# tag = data.tag.value_counts()
# display(tag)

# df = pd.DataFrame(tag)
# # df.head()
# #Save
# df.to_csv('tag.csv', sep=',',header=df.columns.tolist())